In [ ]:
!pip install pennylane

In [ ]:
import pennylane as qml
import math
import numpy as np
from scipy.linalg import block_diag

In [ ]:
# defing parameters
wires = 5
depth = 4

In [ ]:
dev = qml.device("default.qubit", wires = wires)

<default.qubit device (wires=3) at 0x7b777ba3a1a0>

In [ ]:
# Set parameters
delta = 1.0
Omega = 1.0
gamma = 1.0
Delta_t = 0.01

In [ ]:
# Exact vector a
alpha = np.sqrt(2)/2*delta*Delta_t
beta = (1-gamma*Delta_t/2)

In [ ]:
# Pauli operators
I = np.eye(2)
X = np.array([[0,1],[1,0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
Hadamard = np.array([[1, 1], [1, -1]]) / np.sqrt(2)

In [ ]:
# The non-Hermitian operator H

H0 = np.kron(I,1j*Hadamard)
H1 = np.kron(I,-Z)
H2 = np.kron((-1j * alpha * Hadamard + beta * I) / np.sqrt(alpha**2 + beta**2), I)
H3 = np.kron(-Z,I)
H4 = np.kron(X,X)
H5 = np.kron(X,-1j*Y)
H6 = np.kron(-1j*Y,X)
H7 = np.kron(-Y,Y)

In [ ]:
# The non-Hermitian operator Q

I_4 = np.eye(4)

Q0 = I_4 + Delta_t*H0
Q1 = I_4 + Delta_t*H1
Q2 = I_4 + Delta_t*H2
Q3 = I_4 + Delta_t*H3
Q4 = I_4 + Delta_t*H4
Q5 = I_4 + Delta_t*H5
Q6 = I_4 + Delta_t*H6
Q7 = I_4 + Delta_t*H7

In [ ]:
# Multi-controlled unitary
Lambda_Q = block_diag(Q0, Q1, Q2, Q3, Q4, Q5, Q6, Q7)

In [ ]:
def Uq(theta):
  @qml.qnode(dev)
  def circuit(theta):
    UqLay(theta[0:20])
    UqLay(theta[20:40])
    UqLay(theta[40:60])
    UqLay(theta[60:80])

    return qml.state()

  state = circuit(theta)

  # UqFunction = abs(1-1/32**2)*trace((Uq(theta)')*Lambda_Q))**2)
  return np.abs(1-1/32**2)*(np.trace(state)**2)


In [ ]:
#UqLay
def UqLay(theta, wires):
  @qml.qnode(dev)
  def circuit():
    for i in range(wires):
      qml.RZ(theta[i], wires = i)
      qml.RY(theta[i+5], wires = i)
      qml.RZ(theta[i+10], wires = i)

      qml.CRY(theta[16], wires = [0,1])
      qml.CRY(theta[17], wires = [1,2])
      qml.CRY(theta[18], wires = [2,3])
      qml.CRY(theta[19], wires = [3,4])

      return qml.state()

    return circuit()